In [1]:
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

## Using Tesla and Inflation together to create Vader sentiment

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")


In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)


In [5]:
# Fetch all the news about Tesla and Inflation
tesla_headlines = newsapi.get_everything(
    q="tesla AND inflation",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Tesla and inflation: {tesla_headlines['totalResults']}")

# Show sample article
tesla_headlines["articles"][0]

Total articles about Tesla and inflation: 476


{'source': {'id': None, 'name': 'New York Times'},
 'author': 'The New York Times',
 'title': 'Here is where inflation stands as the Fed meets.',
 'description': 'A jump in consumer prices is sure to come up at a news conference on Wednesday after the Federal Reserve’s meeting.',
 'url': 'https://www.nytimes.com/live/2021/06/16/business/economy-stock-market-news',
 'urlToImage': 'https://static01.nyt.com/images/2021/06/16/business/16economy-briefing-FedInflation/merlin_188586378_b7a0c048-2add-4403-b30c-357399ab4cea-facebookJumbo.jpg',
 'publishedAt': '2021-06-16T10:21:32Z',
 'content': 'A day after ousting two top executives, the electric truck start-up Lordstown Motors said on Tuesday that it was on track to start production in September even if it does not raise additional funding… [+3149 chars]'}

In [6]:
# Create the Tesla Inflation sentiment scores DataFrame
tesla_sentiments = []

for article in tesla_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tesla_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
tesla_df = pd.DataFrame(tesla_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
tesla_df = tesla_df[cols]

tesla_df.head()


,date,text,compound,positive,negative,neutral
0,2021-06-16,"A day after ousting two top executives, the el...",0.2023,0.050,0.000,0.950
1,2021-06-08,(Reuters) - U.S. stock index futures fell on T...,0.7096,0.148,0.000,0.852
2,2021-06-08,"(Repeats with no change to headline, text)\r\n...",-0.2960,0.000,0.062,0.937
3,2021-06-08,A street sign is seen in front of the New York...,0.0000,0.000,0.000,1.000
4,2021-06-15,* Traders watching for clues on timing of Fed ...,0.0000,0.000,0.000,1.000


In [7]:
tesla_df.describe()


,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.114672,0.059040,0.038890,0.902040
std,0.386037,0.054452,0.058124,0.076206
min,-0.817900,0.000000,0.000000,0.663000
25%,-0.109025,0.000000,0.000000,0.865750
50%,0.077200,0.064500,0.000000,0.918500
75%,0.437400,0.089250,0.062000,0.943250
max,0.796400,0.192000,0.273000,1.000000
